In [ ]:
!nvidia-smi

Sun Jan  5 12:55:03 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8              14W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# !pip install -U bitsandbytes

In [1]:
import torch

In [2]:
# import transformers

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [4]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [ ]:
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    load_in_8bit=True,  # Quantization for memory efficiency
    torch_dtype=torch.float16,
    device_map="auto"
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
        # quantization_config=quantization_config,
        # torch_dtype=torch.float16,
        # device_map="auto",
        # trust_remote_code=True

In [ ]:
# attn_implementation="flash_attention_2"

In [ ]:
# Format the prompt according to Mistral's chat template
prompt = """<s>[INST] Explain how to bake a sourdough to me in a nutshell. [/INST]"""

# Tokenize and generate
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens=1000,
    temperature=0.7,
    do_sample=True,
    use_cache=True,      # Enable KV-caching
    num_beams=1
)

# Decode the response
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Explain how to bake a sourdough to me in a nutshell. [/INST] To bake a sourdough, you will need the following ingredients:

* Flour (preferably bread, but all-purpose will work)
* Water
* Yeast (optional, but highly recommended)
* Salt

Here is a basic recipe:

1. Mix together the flour, water, and yeast in a large bowl.
2. Let the mixture sit for a few hours or overnight, allowing the yeast to ferment and create bubbles.
3. Once the mixture has risen, add salt and knead the dough until it becomes smooth and elastic.
4. Shape the dough into a loaf and place it in a baking dish.
5. Let the dough rise again for another 30 minutes or so.
6. Preheat your oven to 425°F (220°C).
7. Use a sharp knife to make shallow slashes on the top of the dough.
8. Bake the sourdough for 25-30 minutes, or until the crust is golden brown and the bread sounds hollow when tapped on the bottom.
9. Let the sourdough cool for a few hours before slicing and serving.

Note that the specifics of the recipe m

In [6]:
prompt = """<s>[INST] Explain how to bake a sourdough to me in a nutshell. [/INST]"""

In [9]:
tokenizer.encode(prompt)

[1,
 1,
 28792,
 16289,
 28793,
 13702,
 426,
 910,
 298,
 287,
 621,
 264,
 18193,
 28715,
 900,
 298,
 528,
 297,
 264,
 21204,
 13436,
 28723,
 733,
 28748,
 16289,
 28793]

In [6]:
# Demonstrate token label alignment

In [7]:
# Example conversation and response
conversation = "Speaker 1: Maybe call back in about an hour's time?\nSpeaker 0: Okay, no problem. I will send you some email."
response = "- Action: Call back the client. When: in about an hour's time. By Who: Agent"

In [8]:
# Create full prompt as we do in the dataset
instruction = (
    "Extract action items from the following conversation. "
    "Format each action item as:\n"
    "- Action: <detail action>. When: <detail required due time>. By Who: <specify by whom>\n"
    f"Conversation:\n{conversation}"
)
print(instruction)

Extract action items from the following conversation. Format each action item as:
- Action: <detail action>. When: <detail required due time>. By Who: <specify by whom>
Conversation:
Speaker 1: Maybe call back in about an hour's time?
Speaker 0: Okay, no problem. I will send you some email.


In [9]:
full_prompt = f"<s>[INST] {instruction} [/INST]\n{response}</s>"
print(full_prompt)

<s>[INST] Extract action items from the following conversation. Format each action item as:
- Action: <detail action>. When: <detail required due time>. By Who: <specify by whom>
Conversation:
Speaker 1: Maybe call back in about an hour's time?
Speaker 0: Okay, no problem. I will send you some email. [/INST]
- Action: Call back the client. When: in about an hour's time. By Who: Agent</s>


In [10]:
# Tokenize
encoded = tokenizer(full_prompt, return_tensors='pt')

In [12]:
encoded['input_ids']

tensor([[    1,     1, 28792, 16289, 28793,  1529,  2107,  2992,  4907,   477,
           272,  2296,  7114, 28723, 18748,  1430,  2992,  2515,   390, 28747,
            13, 28733,  9624, 28747,   523, 16017,  2992, 13902,  1684, 28747,
           523, 16017,  3030,  2940,   727, 13902,  2463,  6526, 28747,   523,
          4101,  1575,   486,  6105, 28767,    13,   856, 25422, 28747,    13,
         24812,  4776, 28705, 28740, 28747,  5833,  1034,   852,   297,   684,
           396,  5115, 28742, 28713,   727, 28804,    13, 24812,  4776, 28705,
         28734, 28747, 19811, 28725,   708,  2700, 28723,   315,   622,  4080,
           368,   741,  4927, 28723,   733, 28748, 16289, 28793,    13, 28733,
          9624, 28747,  5529,   852,   272,  3698, 28723,  1684, 28747,   297,
           684,   396,  5115, 28742, 28713,   727, 28723,  2463,  6526, 28747,
         23393,     2]])

In [13]:
encoded['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [14]:
# Get token IDs and decode them individually to show the alignment
input_ids = encoded['input_ids'][0]

In [15]:
input_ids

tensor([    1,     1, 28792, 16289, 28793,  1529,  2107,  2992,  4907,   477,
          272,  2296,  7114, 28723, 18748,  1430,  2992,  2515,   390, 28747,
           13, 28733,  9624, 28747,   523, 16017,  2992, 13902,  1684, 28747,
          523, 16017,  3030,  2940,   727, 13902,  2463,  6526, 28747,   523,
         4101,  1575,   486,  6105, 28767,    13,   856, 25422, 28747,    13,
        24812,  4776, 28705, 28740, 28747,  5833,  1034,   852,   297,   684,
          396,  5115, 28742, 28713,   727, 28804,    13, 24812,  4776, 28705,
        28734, 28747, 19811, 28725,   708,  2700, 28723,   315,   622,  4080,
          368,   741,  4927, 28723,   733, 28748, 16289, 28793,    13, 28733,
         9624, 28747,  5529,   852,   272,  3698, 28723,  1684, 28747,   297,
          684,   396,  5115, 28742, 28713,   727, 28723,  2463,  6526, 28747,
        23393,     2])

In [16]:
# Print each position's input and what should be predicted next
for i in range(len(input_ids) - 1):  # -1 because last token doesn't predict anything
    current_token = tokenizer.decode(input_ids[i])
    next_token = tokenizer.decode(input_ids[i + 1])
    print(f"{i:^8} | {current_token:20} → {next_token:20}")

   0     | <s>                  → <s>                 
   1     | <s>                  → [                   
   2     | [                    → INST                
   3     | INST                 → ]                   
   4     | ]                    → Ex                  
   5     | Ex                   → tract               
   6     | tract                → action              
   7     | action               → items               
   8     | items                → from                
   9     | from                 → the                 
   10    | the                  → following           
   11    | following            → conversation        
   12    | conversation         → .                   
   13    | .                    → Format              
   14    | Format               → each                
   15    | each                 → action              
   16    | action               → item                
   17    | item                 → as                  
   18    |

In [45]:
def find_token_sequence_position(input_ids, target_sequence):
    # Get the actual tokens we want to find - just INST and ]
    inst_middle_end = target_sequence[1:]  # Skip the first token (the problematic '[')
    input_ids_list = input_ids.tolist()

    # Find where these tokens appear consecutively
    for i in range(len(input_ids_list) - len(inst_middle_end) + 1):
        if input_ids_list[i:i + len(inst_middle_end)] == inst_middle_end:
            # Return the position of the '[' that comes before these tokens
            return i - 1
    return -1

# Usage in your code:
inst_tokens = tokenizer.encode('[INST]', add_special_tokens=False)
end_inst_tokens = tokenizer.encode('[/INST]', add_special_tokens=False)

instruction_start = find_token_sequence_position(input_ids, inst_tokens)
instruction_end = find_token_sequence_position(input_ids, end_inst_tokens)

In [46]:
instruction_start

2

In [47]:
instruction_end

84

In [48]:
# Let's see what we have vs what we're looking for
print("What we're looking for:")
print("'[INST]' tokens:", tokenizer.encode('[INST]', add_special_tokens=False))
print("Token strings:", tokenizer.convert_ids_to_tokens(tokenizer.encode('[INST]', add_special_tokens=False)))

print("\nWhat's in our input_ids:")
print("First 10 tokens:", input_ids[:10].tolist())
print("Token strings:", tokenizer.convert_ids_to_tokens(input_ids[:10]))

What we're looking for:
'[INST]' tokens: [733, 16289, 28793]
Token strings: ['▁[', 'INST', ']']

What's in our input_ids:
First 10 tokens: [1, 1, 28792, 16289, 28793, 1529, 2107, 2992, 4907, 477]
Token strings: ['<s>', '<s>', '[', 'INST', ']', '▁Ex', 'tract', '▁action', '▁items', '▁from']


In [ ]:
# The tokenzizr handles the '[' differently if it is a stand-alone (733) or a part of the INST (28792)

In [35]:
tokenizer.encode('[INST]', add_special_tokens=False )

[733, 16289, 28793]

In [52]:
tokenizer.encode('<s>[INST] ', add_special_tokens=False )

[1, 28792, 16289, 28793, 28705]